[Node 43: Eingebauter Profiler](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node43.html)

Navigation:

**Next:** [Line Profiler](node44.ipynb) **Up:** [Profiling von Programmen](node40.ipynb) **Previous:** [Einfacher Timer](node42.ipynb)

##  Eingebauter Profiler
Das eingebaute [profile](https://docs.python.org/3/library/profile.html)-Modul zeichnet alle Funktionsaufrufe und deren Ausführzeit auf. Ein einfaches Beispiel demonstriert die Funktionsweise:

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

Die Bedeutung der einzelnen Spalten ist in der [Dokumentation](https://docs.python.org/3/library/profile.html#instant-user-s-manual) erklärt. Insbesondere:
  * `ncalls`: Anzahl Aufrufe
  * `tottime`: gesamte Zeit ohne Unterfunktionen
  * `cumtime`: gesamte Zeit inklusive Unterfunktionen

(Hinweis: Beim zweiten Aufruf ist die Liste wesentlich kürzer, da `re.compile` ein [eingebautes Caching](https://docs.python.org/3/library/re.html#re.compile) hat.)

---

Der Profiler kann auch auf der Kommandozeile für ein vollständiges Programm angeschaltet werden:
```bash
python -m cProfile myscript.py
```

bzw. die Ausgabe in eine Datei umgeleitet werden, die dann mit Tool `gprof2dot` graphisch aufbereitet werden kann:
```bash
python -m cProfile -o myscript.profile myscript.py
```

Zur Visualisierung lädt man das Skript `gprof2dot.py` mit folgendem Befehl herunter:
```bash
git clone https://github.com/jrfonseca/gprof2dot
```

 und erzeugt ein Ablaufdiagramm in der Datei myscript.pdf mit folgendem Befehl:
```bash
python gprof2dot/gprof2dot.py -f pstats myscript.profile | dot -Tpdf -o myscript.pdf
```

Ein Beispieldiagramm für das Skript
```python
import re
re.compile("foo|bar")
```
sieht folgendermaßen aus:
![Image script](figures/myscript.svg "Image script")

Ein etwas ausführlicheres Beispiel, da Profiling mit Hilfe eines Dekorierers einschaltet und eine formatierte Ausgabe erzeugt, ist in folgendem Programmbeispiel gegeben:

In [ ]:
import cProfile
def do_cprofile(func):
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            res = func(*args, **kwargs)
            profile.disable()
            return res
        finally:
            profile.print_stats()
    return profiled_func

def get_number():
    for x in range(5000000):
        yield x

@do_cprofile
def teure_funktion():
    for x in get_number():
        i = x ^ x ^ x
    return 'Ergebnis'

# profiling
result = teure_funktion()

Man erkennt, welche Funktion langsam ist, aber nicht den eigentlichen Grund.  